In [6]:
from typing import Dict, Union
from IPython import get_ipython
import autogen

class IPythonUserProxyAgent(autogen.UserProxyAgent):
    def generate_init_message(self, *args, **kwargs) -> Union[str, Dict]:
        return super().generate_init_message(*args, **kwargs) + """If you suggest code, the code will be executed in IPython."""
    def run_code(self, code, **kwargs):
        result = self._ipython.run_cell("%%capture --no-display cap\n" + code)
        log = self._ipython.ev("cap.stdout")
        log += self._ipython.ev("cap.stderr")
        if result.result is not None:
            log += str(result.result)
        exitcode = 0 if result.success else 1
        if result.error_before_exec is not None:
            log += f"\n{result.error_before_exec}"
            exitcode = 1
        if result.error_in_exec is not None:
            log += f"\n{result.error_in_exec}"
            exitcode = 1
        return exitcode, log, None
    

In [10]:
import autogen

from autogen import AssistantAgent, UserProxyAgent,oai
from typing import Dict, Union
from IPython import get_ipython
import autogen
import subprocess
config_list = [
    {
        "model": "codellama-7b-instruct.Q5_K_M.gguf",
        "api_base": "http://127.0.0.1:8000/v1",
        "api_type": "open_ai",
        "api_key": "NULL",
    }
]

llm_config = {"config_list": config_list, "seed": 42, "request_timeout": 2028,}

class IPythonUserProxyAgent(UserProxyAgent):
    def generate_init_message(self, *args, **kwargs) -> Union[str, Dict]:
        return super().generate_init_message(*args, **kwargs) + \
               """If you suggest code, the code will be executed."""

    def run_code(self, code, **kwargs):
        try:
            # Execute the code using Python subprocess
            result = subprocess.run(['python', '-c', code],
                                    capture_output=True,
                                    text=True,
                                    check=True)
            # Return exit code, stdout, and stderr
            return (result.returncode, result.stdout, result.stderr, None)
        except subprocess.CalledProcessError as e:
            # Handle errors in execution
            return (e.returncode, e.stdout, e.stderr, None)

# Create assistant agent
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    system_message="""Engineer. You follow an approved plan. 
    You write python/shell code to solve tasks. 
    Wrap the code in a code block that specifies the script type. 
    The user can't modify your code. So do not suggest incomplete 
    code which requires others to modify. Don't use a code block 
    if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. Do not ask 
    others to copy and paste the result. Check the execution result 
    returned by the executor. If the result indicates there is 
    an error, fix the error and output the code again. Suggest
    the full code instead of partial code or code changes. If
    the error can't be fixed or if the task is not solved even
    after the code is executed successfully, analyze the 
    problem, revisit your assumption, collect additional info
    you need, and think of a different approach to try.
""",
)

ipy_user = IPythonUserProxyAgent(
    "ipython_user_proxy",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get(
    "content", "").rstrip().endswith(
    "TERMINATE") or x.get("content", "").rstrip().endswith('"TERMINATE".'),
)
# the assistant receives a message from the user, which contains the task description
ipy_user.initiate_chat(
    assistant,
    message="""plot a stacked area chart
     visualizing the yearly high and low 
     prices of Apple (AAPL), Google 
     (GOOGL), Amazon (AMZN), and Microsoft (MSFT)
    use yfinance,matplotlib,pandas packages
    """,
)

ipython_user_proxy (to assistant):

plot a stacked area chart
     visualizing the yearly high and low 
     prices of Apple (AAPL), Google 
     (GOOGL), Amazon (AMZN), and Microsoft (MSFT)
    use yfinance,matplotlib,pandas packages
    If you suggest code, the code will be executed.

--------------------------------------------------------------------------------
assistant (to ipython_user_proxy):

  To plot a stacked area chart using the yfinance, matplotlib, and pandas packages in Python, you can follow these steps:
```
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

# Get historical stock prices for Apple (AAPL), Google (GOOGL), Amazon (AMZN), and Microsoft (MSFT)
stock_prices = yf.download('AAPL GOOGL AMZN MSFT', start='2015-01-01', end='2022-01-01')['Adj Close']

# Convert the data to a Pandas DataFrame
df = pd.DataFrame(stock_prices)

# Set the index to the date column
df.set_index('Date', inplace=True)

# Calculate the yearly high and low prices for

ValueError: too many values to unpack (expected 3)

In [14]:
from autogen import AssistantAgent, UserProxyAgent,oai, config_list_openai_aoai, ChatCompletion

# Configuration for the LLaMA model
config_list = [
    {
        "model": "codellama-7b-instruct.Q5_K_M.gguf",
        "api_base": "http://127.0.0.1:8000/v1",
        "api_type": "open_ai",
        "api_key": "NULL",
    }
]

llm_config = {"config_list": config_list, "cache_seed": 42, "request_timeout": 2028, "temperature": 0.7}

# Configuration for code execution
code_execution_config = {
    "work_dir": "planning",  # Specify the working directory
    "use_docker": False       # Use Docker for code execution
}
planner = AssistantAgent(
    name="planner",
    llm_config={"config_list": config_list},
    # the default system message of the AssistantAgent is overwritten here
    system_message="""You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. 
    Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step that can be implemented by writing code. 
    For example, browsing the web can be implemented by writing code that reads and prints the content of a web page. Finally, inspect the execution result. 
    Shell code should be formatted as `sh`
    If the plan is not good, suggest a better plan. If the execution is wrong, analyze the error and suggest a fix."""
)
planner_user = UserProxyAgent(
    name="planner_user",
    llm_config=llm_config,
    max_consecutive_auto_reply=0,  # terminate without auto-reply
    human_input_mode="NEVER",
)

def ask_planner(message):
    planner_user.initiate_chat(planner, message=message)
    # return the last message received from the planner
    return planner_user.last_message()["content"]


# create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    llm_config=llm_config
)

# create a UserProxyAgent instance named "user_proxy"
# Modify the code_execution_config
user_proxy = UserProxyAgent(
    name="user_proxy",
    llm_config=llm_config,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config=code_execution_config,
    function_map={"ask_planner": ask_planner},
)


ChatCompletion.start_logging( compact=False)

# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------
assistant (to user_proxy):

  To get the current date, you can use the `datetime` module in Python:
```
from datetime import datetime

now = datetime.now()
print(now)
```
This will output the current date and time in your local timezone.

To compare the year-to-date gain for META and TESLA, you can use the `pandas` library to read in historical stock data from a CSV file or other source, then calculate the year-to-date gain for each company using the `dt` column (which represents the date) and the `Close` column (which represents the closing price).
```
import pandas as pd

# Read in historical stock data from a CSV file
df = pd.read_csv('stock_data.csv')

# Calculate year-to-date gain for META
meta_ytd_gain = df[df['Symbol'] == 'META']['Close'].iloc[-1] - df[df['Symbol'] == 'META']['Close'].iloc[0]
print(f'Y

KeyboardInterrupt: 

In [15]:
import autogen

# Configuration for the LLaMA model
config_list = [
    {
        "model": "codellama-7b-instruct.Q5_K_M.gguf",
        "api_base": "http://127.0.0.1:8000/v1",
        "api_type": "open_ai",
        "api_key": "NULL",
    }
]

# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 42,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
)
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""Plot a chart of their stock price change YTD and save to stock_price_ytd.png.""",
)

user_proxy (to assistant):

Plot a chart of their stock price change YTD and save to stock_price_ytd.png.

--------------------------------------------------------------------------------
assistant (to user_proxy):

  To plot a chart of the stock price change YTD and save it as a PNG file, you can use the `matplotlib` library in Python. Here's an example code that you can use:
```python
import matplotlib.pyplot as plt

# Load the data from the CSV file
data = pd.read_csv('stock_price_ytd.csv')

# Plot the chart
plt.plot(data['Date'], data['Close'])
plt.title('Stock Price Change YTD')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.savefig('stock_price_ytd.png')
```
This code assumes that the stock price data is stored in a CSV file named `stock_price_ytd.csv` and that the date column is named `Date` and the close price column is named `Close`. You can modify the code to match your specific data format.

After running this code, you should see a chart of the stock price change YTD save